In [ ]:
# Initial operation before action on dataframe
# Import library
import pandas as pd
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox
from openpyxl import load_workbook
from openpyxl.worksheet.table import Table

def trial_execute(input_file, sheet_name):
    # Import file
    excel_file = pd.ExcelFile(input_file)
    df = excel_file.parse(sheet_name, header=None)
    
    #=================================================================================================
    # Operation on datafram
    # Delete row 1-13
    df = df.iloc[13:].reset_index(drop=True)
    
    # Set 1st row as header
    df.columns = df.iloc[0]
    df = df.iloc[0:].reset_index(drop=True)
    
    # Delete column A, I–AZ, BX–end
    columns = df.columns.tolist()
    total_cols = len(columns)
    drop_indexes = [0]  
    drop_indexes += list(range(8, 71)) 
    if total_cols > 75:
        drop_indexes += list(range(75, total_cols)) 
    df = df.drop(df.columns[drop_indexes], axis=1)
    
    # Import Member list
    member_file = os.path.join("Require", "Members.xlsx")
    member_df = pd.read_excel(member_file, sheet_name="Member")
    
    # Create Designation column for matching
    designation_set = set(member_df["Designation"].dropna().astype(str))
    
    # Create column next to 1st column
    df.insert(1, "Matched", None)
    
    # Match value from 1st column, then fill in the 2 rows before
    for i in range(2, len(df)):
        value = str(df.iloc[i, 0]).strip()
        if value in designation_set:
            df.iloc[i - 2, 1] = value
    
    # Set column name
    df.columns = [
        "No", "PIC", "Size", "Pattern", "Purpose",
        "Confidential", "Reference", "Owner",
        "Product code", "Optional code", "PC code"
    ]
    
    # Create column Project next to column Purpose
    purpose_idx = df.columns.get_loc("Purpose")
    df.insert(purpose_idx + 1, "Project", None)
    
    # Fill value in column Project from the value in the next row on column Purpose
    for i in range(len(df) - 1):
        if str(df.at[i, "PIC"]).strip().lower() not in ["", "none"]:
            df.at[i, "Project"] = df.at[i + 1, "Purpose"]
    
    # Create column Spec next to column Project 
    project_idx = df.columns.get_loc("Project")
    df.insert(project_idx + 1, "Spec", None)
    
    # Fill value in column Spec from the value in the next 2 rows on column Purpose
    for i in range(len(df) - 2):
        if str(df.at[i, "PIC"]).strip().lower() not in ["", "none"]:
            df.at[i, "Spec"] = df.at[i + 2, "Purpose"]
    
    # Create column Designer next to column Owner
    designer_idx = df.columns.get_loc("Owner")
    df.insert(designer_idx + 1, "Designer", None)
    
    # Fill value in column Designer from the value in the next row on column Owner
    for i in range(len(df) - 1):
        if str(df.at[i, "PIC"]).strip().lower() not in ["", "none"]:
            df.at[i, "Designer"] = df.at[i + 1, "Owner"]
    
    # Create column Production code next to column PC code 
    production_idx = df.columns.get_loc("PC code")
    df.insert(production_idx + 1, "Production code", None)
    
    # Fill value in column Production code from the value in the next row on column PC code
    for i in range(len(df) - 1):
        if str(df.at[i, "PIC"]).strip().lower() not in ["", "none"]:
            df.at[i, "Production code"] = df.at[i + 1, "PC code"]
    
    # Delete row where the value in Production code is blank
    df = df[df["Production code"].notna() & (df["Production code"].astype(str).str.strip() != "")]
    
    # Create column AppUser next to column PIC 
    members_df = pd.read_excel(member_file, sheet_name="Member")
    designation_to_user = dict(zip(members_df["Designation"].astype(str), members_df["App_username"]))
    
    pic_idx = df.columns.get_loc("PIC")
    df.insert(pic_idx + 1, "AppUser", None)
    df["AppUser"] = df["PIC"].astype(str).map(designation_to_user)
    
    # Create excel
    output_save = os.path.join("Require", "Trial information.xlsx")
    df.to_excel(output_save, index=False)
    
    #=================================================================================================
    # Load workbook and worksheet 
    df = pd.read_excel(output_save)
    wb = load_workbook(output_save)
    ws = wb.active
                
    # Create Table 
    table_name = "Trial_Information"
    start_cell = "A1"
    end_column = chr(65 + len(df.columns) - 1)  
    end_row = df.shape[0] + 1  
    table_range = f"{start_cell}:{end_column}{end_row}"
    
    table = Table(displayName=table_name, ref=table_range)
    
    # Save excel
    ws.add_table(table)
    wb.save(output_save)

#==============================================================================================================================
#Run GUI
def run_gui():
    root = tk.Tk()
    root.title("Trial Plan Processing")
    root.geometry("400x200")

    def run_processing():
        try:
            file_path = filedialog.askopenfilename(filetypes = [("Excel files", "*.xlsx")])
            if not file_path:
                return

            xl = pd.ExcelFile(file_path)
            sheet_name = xl.sheet_names[0]

            trial_execute(file_path, sheet_name)

            output_path = filedialog.asksaveasfilename(
                defaultextension = ".xlsx",
                initialfile = "Trial information.xlsx"
            )

            output_save = os.path.join("Require", "Trial information.xlsx")
            
            if output_path:
                shutil.copy(output_save, output_path)
                messagebox.showinfo("Done!", f"Your file has been saved at:\{output_path}")
                os.remove(output_save)

        except Exception as e:
            messagebox.showerror("Error!", str(e))

    tk.Label(root, text = "Upload Trial Plan File!", font = ("TH SarabunPSK", 14)).pack(pady = 20)
    tk.Button(root, text = "Upload", command = run_processing, font = ("TH SarabunPSK", 14)).pack(pady = 10)
    
    root.mainloop()

if __name__ == "__main__":
    run_gui()
